In [6]:
# Copyright (c) 2012, Ryan Gomba
# All rights reserved.
# 
# Redistribution and use in source and binary forms, with or without
# modification, are permitted provided that the following conditions are met:
# 
# 1. Redistributions of source code must retain the above copyright notice, this
#    list of conditions and the following disclaimer. 
# 2. Redistributions in binary form must reproduce the above copyright notice,
#    this list of conditions and the following disclaimer in the documentation
#    and/or other materials provided with the distribution.
# 
# THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND
# ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED
# WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE
# DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT OWNER OR CONTRIBUTORS BE LIABLE FOR
# ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES
# (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES;
# LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND
# ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT
# (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS
# SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.
# 
# The views and conclusions contained in the software and documentation are those
# of the authors and should not be interpreted as representing official policies, 
# either expressed or implied, of the FreeBSD Project.

import math
import json
import numpy as np
import functools
import pickle
################################################################################
# POINT
################################################################################

class Point:
    
    def __init__(self, latitude, longitude):
        
        self.latitude = latitude
        self.longitude = longitude
        self.cd = None              # core distance
        self.rd = None              # reachability distance
        self.processed = False      # has this point been processed?
        
    # --------------------------------------------------------------------------
    # calculate the distance between any two points on earth
    # --------------------------------------------------------------------------
    
    def __eq__(self, other):
        if self.latitude == other.latitude and self.longitude==other.longitude:
            return True
        else:
            return False
        
    def distance(self, point):
        
        # convert coordinates to radians
        
        p1_lat, p1_lon, p2_lat, p2_lon = [math.radians(c) for c in
            [self.latitude, self.longitude, point.latitude, point.longitude]]
        
        numerator = math.sqrt(
            math.pow(math.cos(p2_lat) * math.sin(p2_lon - p1_lon), 2) +
            math.pow(
                math.cos(p1_lat) * math.sin(p2_lat) -
                math.sin(p1_lat) * math.cos(p2_lat) *
                math.cos(p2_lon - p1_lon), 2))

        denominator = (
            math.sin(p1_lat) * math.sin(p2_lat) +
            math.cos(p1_lat) * math.cos(p2_lat) *
            math.cos(p2_lon - p1_lon))
        
        # convert distance from radians to meters
        # note: earth's radius ~ 6372800 meters
        
        return math.atan2(numerator, denominator) * 6372800
        
    # --------------------------------------------------------------------------
    # point as GeoJSON
    # --------------------------------------------------------------------------
        
    def to_geo_json_dict(self, properties=None):
        
        return {
            'type': 'Feature',
            'geometry': {
                'type': 'Point',
                'coordinates': [
                    self.longitude,
                    self.latitude,
                ]
            },
            'properties': properties,
        }
 
    def __repr__(self):
        return '(%f, %f)' % (self.latitude, self.longitude)

################################################################################
# CLUSTER
################################################################################

class Cluster:
    
    def __init__(self, points):
        
        self.points = points
        self.center = 0
        self.radius = 0
        
    # --------------------------------------------------------------------------
    # calculate the centroid for the cluster
    # --------------------------------------------------------------------------

    def centroid(self):
        
        return Point(sum([p.latitude for p in self.points])/len(self.points),
            sum([p.longitude for p in self.points])/len(self.points))
            
    # --------------------------------------------------------------------------
    # calculate the region (centroid, bounding radius) for the cluster
    # --------------------------------------------------------------------------
    
    def region(self):
        
        centroid = self.centroid()
        #radius = functools.reduce(lambda p1, p2: p1 if p1.distance(centroid) > p2.distance(centroid) else p2, self.points).distance(centroid)
        dis = 0
        for p in self.points:
            dis += p.distance(centroid)
        radius = dis/(len(self.points)-1) 
        return centroid, radius
        
    # --------------------------------------------------------------------------
    # cluster as GeoJSON
    # --------------------------------------------------------------------------
        
    def to_geo_json_dict(self, user_properties=None):
        
        self.center, self.radius = self.region()
        properties = { 'radius': self.radius }
        if user_properties: properties.update(user_properties)
        
        return {
            'type': 'Feature',
            'geometry': {
                'type': 'Point',
                'coordinates': [
                    self.center.longitude,
                    self.center.latitude,
                ]
            },
            'properties': properties,
        }

################################################################################
# OPTICS
################################################################################

class Optics:
    
    def __init__(self, points, max_radius, min_cluster_size):
        
        self.points = points
        self.max_radius = max_radius                # maximum radius to consider
        self.min_cluster_size = min_cluster_size    # minimum points in cluster
        self.rd = []
    
    # --------------------------------------------------------------------------
    # get ready for a clustering run
    # --------------------------------------------------------------------------
    
    def _setup(self):
        
        for p in self.points:
            p.rd = None
            p.processed = False
        self.unprocessed = [p for p in self.points]
        self.ordered = []

    # --------------------------------------------------------------------------
    # distance from a point to its nth neighbor (n = min_cluser_size)
    # --------------------------------------------------------------------------
    
    def _core_distance(self, point, neighbors):

        if point.cd is not None: return point.cd
        if len(neighbors) >= self.min_cluster_size - 1:
            sorted_neighbors = sorted([n.distance(point) for n in neighbors])
            point.cd = sorted_neighbors[self.min_cluster_size - 2]
            return point.cd
        
    # --------------------------------------------------------------------------
    # neighbors for a point within max_radius
    # --------------------------------------------------------------------------
    
    def _neighbors(self, point):
        
        return [p for p in self.points if p is not point and
            p.distance(point) <= self.max_radius]
            
    # --------------------------------------------------------------------------
    # mark a point as processed
    # --------------------------------------------------------------------------
        
    def _processed(self, point):
    
        point.processed = True
        self.unprocessed.remove(point)
        self.ordered.append(point)
    
    # --------------------------------------------------------------------------
    # update seeds if a smaller reachability distance is found
    # --------------------------------------------------------------------------

    def _update(self, neighbors, point, seeds):
        
        # for each of point's unprocessed neighbors n...

        for n in [n for n in neighbors if not n.processed]:
            
            # find new reachability distance new_rd
            # if rd is null, keep new_rd and add n to the seed list
            # otherwise if new_rd < old rd, update rd
            
            new_rd = max(point.cd, point.distance(n))
            if n.rd is None:
                n.rd = new_rd
                seeds.append(n)
            elif new_rd < n.rd:
                n.rd = new_rd
    
    # --------------------------------------------------------------------------
    # run the OPTICS algorithm
    # --------------------------------------------------------------------------

    def run(self):
        
        self._setup()
        
        # for each unprocessed point (p)...
        
        while self.unprocessed:
            point = self.unprocessed[0]
            
            # mark p as processed
            # find p's neighbors
            
            self._processed(point)
            point_neighbors = self._neighbors(point)

            # if p has a core_distance, i.e has min_cluster_size - 1 neighbors

            if self._core_distance(point, point_neighbors) is not None:
                
                # update reachability_distance for each unprocessed neighbor
                
                seeds = []
                self._update(point_neighbors, point, seeds)
                
                # as long as we have unprocessed neighbors...
                
                while(seeds):
                    
                    # find the neighbor n with smallest reachability distance
                    
                    seeds.sort(key=lambda n: n.rd)
                    n = seeds.pop(0)
                    
                    # mark n as processed
                    # find n's neighbors
                    
                    self._processed(n)
                    n_neighbors = self._neighbors(n)
                    
                    # if p has a core_distance...
                    
                    if self._core_distance(n, n_neighbors) is not None:
                        
                        # update reachability_distance for each of n's neighbors
                        
                        self._update(n_neighbors, n, seeds)
                        
        # when all points have been processed
        # return the ordered list      
        self.getRD()

        return self.ordered
        
    # --------------------------------------------------------------------------
    def _score(self):
        rd = []
        for r in self.rd:
            if r is None:
                r = self.max_radius+1 #float('infinity')
                rd.append(r)
            elif r == 0:
                continue
            else:
                rd.append(r)
        self.rd = [r for r in rd]
        rd.sort()
        #print(rd)
        rdth = 0
        min_score = np.inf
        #print(self.ordered)
        N = len(rd)
        for i in range(len(self.rd)):
            if i >= self.min_cluster_size - 1:
                #print(np.std(rd[0:i]),N,i)
                score = np.std(rd[0:i])*(N/i);
                if score < min_score:
                    min_score = score
                    rdth = rd[i]
        #print("rdth: ",rdth)
        return rdth
    
    def cluster(self):
        
        clusters = []
        separators = []
        cluster_threshold = self._score() 

        for i in range(len(self.ordered)):
            this_i = i
            next_i = i + 1
            this_p = self.ordered[i]
            this_rd = this_p.rd if this_p.rd else float('infinity')
            
            # use an upper limit to separate the clusters
            
            if this_rd > cluster_threshold:
                separators.append(this_i)

        separators.append(len(self.ordered))

        for i in range(len(separators) - 1):
            start = separators[i]
            end = separators[i + 1]
            if end - start >= self.min_cluster_size:
                clusters.append(Cluster(self.ordered[start:end]))
        for cluster in clusters:
            cluster.to_geo_json_dict()
        return clusters
    
    def bar(self):
        plt.bar(range(len(self.rd)),self.rd, width=1.0)
        plt.show()
        
    def getRD(self):
        for point in self.ordered:
            self.rd.append(point.rd)
            #print(point.latitude, point.longitude,":",point.rd)
        return self.rd

################################################################################
# POPTICS
################################################################################
class POPTICS:
    def __init__(self, points, yita):
        self.points = points
        self.poptics = Optics(points, 2000, yita)
    
    def run(self):
        self.poptics.run()
        clusters = self.poptics.cluster()  
        return clusters
    
    def show(self):
        self.poptics.bar()

################################################################################
# CLEAN DATA
################################################################################
print("Run our poptics")
#import numpy as np
from collections import defaultdict
#import source.optics as OP

# ------------------------open file--------------------------------#
nyc = open("./nyc.txt", encoding='ISO-8859-1')
#------------------------------------------------------------------#

'''
the dictonary 'user_checkin_history' (defined below) contains inputs to POPTICS
Data format:
{
    user_1 : [(venue_category_1, lantitude_1, longtitude_1), (venue_category_2, lantitude_2, longtitude_2) ...],
    user_2 : [(venue_category_i, lantitude_i, longtitude_i), (venue_category_i+1, lantutide_i+1, longtitude_i+1) ...],
    user_3 : [...],
    ...    :  ... ,
    user_n : [...]
}
'''
#user_checkin_history = dict.fromkeys(range(4), []) #d
#user_checkin_w = dict.fromkeys(range(4), []) #defaultdict([])
user_checkin_history =  defaultdict(list)
user_checkin_w = defaultdict(list)

#---------------------extract data from file-----------------------------#
for line in nyc:
    # index 3 : venue category, 4 : Latitude, 5 : Longitude
    lst = line.split('\t')
    
    # lantitude and longtitude
    lantitude = float(lst[4])
    longtitude = float(lst[5])
    
    # venue category, combine categories into a single category
    category = lst[3]
    if 'Restaurant' in category:
        venue_category = 'Restaurant'
    elif 'Joint' in category:
        venue_category = 'Food Joint'
    elif 'Museum' in category:
        venue_category = 'Museum'
    else:
        venue_category = category
    
    # construct the checkin
    single_checkin = (venue_category, lantitude, longtitude)
    
    # add this checking location, lst[0] is the user's id
    user_checkin_history[lst[0]].append(single_checkin)
#----------------------------------------------------------------------#
#-------------------remove insignificant venues categories and users-----------#
# format : {venue_category_1 : count 1, venue_category_2 : count 2, ...}
venue_count = defaultdict(lambda : 0)
for user, checkins in user_checkin_history.items():
    for checkin in checkins:
        venue_count[checkin[0]] += 1

# delete all checkins whose venue categories have not been visited over 1200 times
for user, checkins in user_checkin_history.items():
    new_checkins = []
    new_w = []
    for checkin in checkins:
        if venue_count[checkin[0]] >= 1200:
            p = Point(checkin[1],checkin[2])
            #print("p outside:",p)
            #print("p outside: ({}, {})".format(p.longitude, p.latitude)) //different precison for different print methods
            if p not in new_checkins:
                new_checkins.append(p)   #remove the category in checkin
                new_w.append(1) 
            else:
                new_w[new_checkins.index(p)] += 1  
                
    user_checkin_history[user] = new_checkins
    user_checkin_w[user] = new_w

# remove users who visited less than 100 places
remove = []
for user, checkins in user_checkin_w.items():
    if sum(checkins) < 100:
        remove.append(user)
for r in remove:
    del user_checkin_history[r]
    
#----------------------REMOVE CHECK-IN NO IN USER'S PATTERN-----------------#
user_pattern = pickle.load( open("user_pattern.pickle", 'rb')) 
pattern_venue = pickle.load( open("pattern_point.pickle", 'rb') )
for user, checkins in user_checkin_history.items():
    new_checkins = []
    for p in checkins:
        if p in pattern_venue[user_pattern[user]]: 
            new_checkins.append(p)  
    user_checkin_history[user] = new_checkins
# print("user pattern cluster")
# pickle.dump(user_checkin_w, open("user_checkin_w.pickle", 'wb'))
# pickle.dump(user_checkin_history, open("user_checkin_no_pattern.pickle", 'wb'))
# user_checkin_history = pickle.load(open("user_checkin_no_pattern.pickle", 'rb'))
################################################################################
# RUN POPTICS
################################################################################
#-------------------CALCULATE THE MINPYS---------------------------------------#
MP = defaultdict(int)
for user in user_checkin_history:
    l = len(user_checkin_history[user])
    MinPts = 0
    if l < 30:
        MinPts = 3
    elif l < 100:
        MinPts = 4
    else:
        MinPts = 5
    MP[user] = MinPts
#--------------------RUN POPTICS FOR EACH USER----------------------------------#    
user_clusters = defaultdict(list)
for user in user_checkin_history:
    print(user)
    POP = POPTICS(user_checkin_history[user] , MP[user])
    user_clusters[user] = POP.run()
print("finish")

Run our poptics
470
69
87
484
642
428
877
691
116
445
895
704
891
901
699
100
443
689
390
32
230
834
962
1032
525
318
1047
738
364
398
839
424
751
889
612
643
718
768
15
458
280
1016
281
276
80
673
337
916
639
752
713
1012
438
217
656
225
628
504
164
235
118
1054
881
706
143
1005
912
537
868
1055
907
303
793
784
734
540
816
623
294
615
173
16
7
592
636
865
621
272
591
976
562
783
319
468
874
107
667
557
725
880
288
335
769
403
573
346
968
539
184
144
71
688
65
885
267
820
516
176
528
30
246
515
710
145
498
894
313
120
798
1078
195
59
662
553
702
84
129
864
658
951
817
371
121
49
50
29
519
340
391
526
645
758
611
577
984
1025
354
373
902
571
440
987
185
559
279
529
131
207
943
179
444
586
510
1044
572
766
162
299
652
315
637
12
787
633
672
973
597
422
365
805
905
940
359
191
130
186
692
1009
806
1076
842
125
474
26
90
989
139
593
328
402
57
221
198
772
408
818
187
242
994
509
969
861
361
296
826
475
149
54
432
386
626
215
17
1013
848
188
434
14
854
985
1083
415
649
208
619
990
722
1042


In [2]:
# pickle.dump(user_clusters, open("user_clusters.pickle", 'wb'))

In [10]:
# print(len(user_clusters))
# a = 0
# for cluster in user_clusters["354"]:
#     for point in cluster.points:
#         print(point)
#         a += 1
# print(a)

In [7]:
###############################################DSR############################################################
import math
import pickle
def gaussian_2d(x, y, mu_x, mu_y, sig):
    exp_value = -(np.power(x-mu_x, 2)+np.power(y-mu_y, 2))/(2*np.power(sig, 2))
    return 1/(np.sqrt(2*np.pi*np.power(sig,2)))*np.exp(exp_value)

user_clusters = pickle.load( open("user_clusters.pickle", 'rb') )
#pattern_venue = pickle.load( open("pattern_point.pickle", 'rb') )

#########################################demand level###########################################################
# venues = [] #[(user1, user2 ...), (user1, user2, ...)]
# venue_user = [] #[venue1(user1, user2 ...), venue2(user1, user2 ...) ...]

venues = []
for user in user_clusters:
    for cluster in user_clusters[user]:
        for p in cluster.points:
            if p not in venues:
                venues.append(p)
            
venue_user = defaultdict(list)
for venue in venues:
    for user in user_clusters:
        for cluster in user_clusters[user]:
            if venue in cluster.points:
                venue_user[venues.index(venue)].append(user)

demand = defaultdict(float)
for venue in venue_user:
    demand[venue] = 0
    for user in venue_user[venue]:
        for cluster in user_clusters[user]:
            if venues[venue] in cluster.points:
                this_center = cluster.center
                this_radius = cluster.radius
                d = gaussian_2d(venues[venue].latitude, venues[venue].longitude, this_center.latitude, this_center.longitude, this_radius)
                demand[venue] += d
                
############################################supply level#######################################################

#convert longitude and latitude to distance
def distance(point1, point2):
    # convert coordinates to radians
    p1_lat, p1_lon, p2_lat, p2_lon = [math.radians(c) for c in
        [point1.latitude, point1.longitude, point2.latitude, point2.longitude]]

    numerator = math.sqrt(
        math.pow(math.cos(p2_lat) * math.sin(p2_lon - p1_lon), 2) +
        math.pow(
            math.cos(p1_lat) * math.sin(p2_lat) -
            math.sin(p1_lat) * math.cos(p2_lat) *
            math.cos(p2_lon - p1_lon), 2))

    denominator = (
        math.sin(p1_lat) * math.sin(p2_lat) +
        math.cos(p1_lat) * math.cos(p2_lat) *
        math.cos(p2_lon - p1_lon))

    # convert distance from radians to meters
    # note: earth's radius ~ 6372800 meters
    return math.atan2(numerator, denominator) * 6372800

#standard deviation
def distance_venue(venue): 
    user_number = 0
    total_dist = 0
    for user in venue_user[venue]: 
        for cluster in user_clusters[user]:
            if venues[venue] in cluster.points:
                u_center = cluster.center
                #print(u_center)
        dist = distance(u_center, venues[venue])
        total_dist += dist
        user_number += 1
    average_dist = total_dist / user_number
    return average_dist

supply = defaultdict(float)
for venue in venue_user:
    supply[venue] = 0
    for user in venue_user[venue]:
        for cluster in user_clusters[user]:
            if venues[venue] in cluster.points:
                this_center = cluster.center
                s = gaussian_2d(venues[venue].latitude, venues[venue].longitude, this_center.latitude, this_center.longitude, distance_venue(venue))
                supply[venue] += s

#################################################demand_supply_ratio#############################################
dsr = []
for i in range(4512):
    dsr.append(demand[i]/supply[i])
    
####################################################display########################################################
dsr_dict = defaultdict(float)
for venue in venue_user:
    dsr_dict[venue] = demand[venue]/supply[venue]

########################################dsr plot section############################################
import numpy as np
import matplotlib.pyplot as plt
x = []
y = []
colors = []
for venue in venues:
    if dsr_dict[venues.index(venue)] < 2.1:
        x.append(venue.longitude)
        y.append(venue.latitude)
        colors.append(dsr_dict[venues.index(venue)])
        
plt.figure(figsize=(20, 20), dpi=80)
plt.scatter(x, y, c=colors, alpha=0.5, vmin=0, vmax=2, cmap='viridis')
plt.colorbar()
plt.savefig("dsr_value.png")


defaultdict(<class 'float'>, {0: 0.0004747471907767433, 1: 0.0004747471907821179, 2: 0.0004747471907669626, 3: 0.000474747190787894, 4: 0.0004747471907774601, 5: 0.0004747471907501846, 6: 0.0002083429665549293, 7: 0.00020834296655557143, 8: 0.0010030859730698757, 9: 0.0006112986863365497, 10: 0.004734384339366375, 11: 0.0022385543264374403, 12: 0.00020834296655819918, 13: 0.00020834296655868837, 14: 0.00020834296655995445, 15: 0.0002083429665565428, 16: 0.0002083429665549931, 17: 0.00020834296655363938, 18: 0.0009608120848565399, 19: 0.0013813701672572667, 20: 0.00020834296655122586, 21: 0.00020834296656144322, 22: 0.00020834296656214684, 23: 0.00020834296656074705, 24: 0.00020834296656360965, 25: 0.00020834296656373864, 26: 0.00020834296656311937, 27: 0.00020834296655651015, 28: 0.0002083429665632038, 29: 0.0002083429665631167, 30: 0.00020834296656200522, 31: 0.00020834296656094676, 32: 0.00020834296656217037, 33: 0.00020834296656016037, 34: 0.0002083429665566446, 35: 0.00020834296655

In [16]:
#################################################demand_supply_ratio#############################################
dsr = []
for i in range(4512):
    dsr.append(demand[i]/supply[i])
print(dsr)

[0.8153347632022783, 0.6045226571712866, 1.081671072761954, 0.29120208556407134, 0.7772333941993168, 1.430036027084648, 1.0198511562957324, 1.014699388454146, 1.5169190156254029, 0.926754794655647, 3.5442496798793157, 2.4678714639642365, 0.7529905107695236, 0.6991642605310209, 0.5945083947159499, 0.790394805298138, 0.8268439406544995, 0.9509920094363135, 0.7316968701667151, 1.019312694369489, 0.9667902601013567, 0.5581805913231966, 0.4753106762028064, 0.5949916787483044, 0.26155414916422315, 0.18683336030451744, 0.277447272944812, 0.7790789343623479, 0.3016071204563031, 0.3388285984133244, 0.5037178874973288, 0.5623839923665385, 0.4797719064867776, 0.6703204865552024, 0.879649893557068, 0.9055726405725236, 1.1756389097745887, 1.2984150846572573, 1.0487210309814188, 1.4747503947544063, 1.0185098551424778, 1.6299929986116182, 1.8196960298412814, 2.0223325630062052, 1.8694180112278154, 2.4131264538783737, 1.4511220267680085, 1.6174351471188404, 1.6794179502095028, 2.161059631960548, 3.985

In [17]:
####################################################display########################################################
dsr_dict = defaultdict(float)
for venue in venue_user:
    dsr_dict[venue] = demand[venue]/supply[venue]
print(dsr_dict)


defaultdict(<class 'float'>, {0: 0.8153347632022783, 1: 0.6045226571712866, 2: 1.081671072761954, 3: 0.29120208556407134, 4: 0.7772333941993168, 5: 1.430036027084648, 6: 1.0198511562957324, 7: 1.014699388454146, 8: 1.5169190156254029, 9: 0.926754794655647, 10: 3.5442496798793157, 11: 2.4678714639642365, 12: 0.7529905107695236, 13: 0.6991642605310209, 14: 0.5945083947159499, 15: 0.790394805298138, 16: 0.8268439406544995, 17: 0.9509920094363135, 18: 0.7316968701667151, 19: 1.019312694369489, 20: 0.9667902601013567, 21: 0.5581805913231966, 22: 0.4753106762028064, 23: 0.5949916787483044, 24: 0.26155414916422315, 25: 0.18683336030451744, 26: 0.277447272944812, 27: 0.7790789343623479, 28: 0.3016071204563031, 29: 0.3388285984133244, 30: 0.5037178874973288, 31: 0.5623839923665385, 32: 0.4797719064867776, 33: 0.6703204865552024, 34: 0.879649893557068, 35: 0.9055726405725236, 36: 1.1756389097745887, 37: 1.2984150846572573, 38: 1.0487210309814188, 39: 1.4747503947544063, 40: 1.0185098551424778, 4

In [18]:
for venue in venues:
    print(venues.index(venue))
    print(venue)

0
(40.685820, -73.991081)
1
(40.687583, -73.989895)
2
(40.683827, -73.992498)
3
(40.694157, -73.991873)
4
(40.697673, -73.993413)
5
(40.702752, -73.990696)
6
(40.723342, -73.982570)
7
(40.722665, -73.988706)
8
(40.721200, -73.988682)
9
(40.724367, -73.987472)
10
(40.722186, -73.997124)
11
(40.725929, -73.996395)
12
(40.729752, -73.999627)
13
(40.731383, -74.000286)
14
(40.733622, -73.999798)
15
(40.731807, -74.003580)
16
(40.734068, -74.006369)
17
(40.728899, -74.005095)
18
(40.737037, -73.996946)
19
(40.731129, -73.993531)
20
(40.734413, -74.010000)
21
(40.730972, -73.985514)
22
(40.732537, -73.985330)
23
(40.731165, -73.982822)
24
(40.735957, -73.987126)
25
(40.739686, -73.985164)
26
(40.740429, -73.995660)
27
(40.732690, -73.974638)
28
(40.743846, -73.994151)
29
(40.745254, -73.985697)
30
(40.748507, -73.986426)
31
(40.748098, -73.982116)
32
(40.747980, -73.992745)
33
(40.751027, -73.994369)
34
(40.753461, -73.979895)
35
(40.752248, -73.976429)
36
(40.757666, -73.976038)
37
(40.7612

783
(40.857552, -73.856359)
784
(40.857545, -73.856363)
785
(40.857415, -73.856492)
786
(40.743778, -73.881505)
787
(40.743525, -73.881804)
788
(40.743318, -73.881930)
789
(40.743091, -73.882200)
790
(40.742950, -73.882286)
791
(40.742758, -73.882315)
792
(40.743815, -73.881379)
793
(40.742969, -73.882767)
794
(40.756165, -73.990602)
795
(40.756221, -73.990834)
796
(40.756505, -73.990966)
797
(40.756800, -73.990966)
798
(40.756263, -73.991247)
799
(40.750963, -73.990817)
800
(40.750248, -73.989916)
801
(40.749990, -73.990208)
802
(40.749859, -73.989962)
803
(40.749955, -73.987556)
804
(40.718201, -74.043356)
805
(40.718226, -74.043820)
806
(40.719487, -74.042937)
807
(40.720044, -74.042867)
808
(40.718753, -74.045690)
809
(40.718094, -74.047119)
810
(40.719860, -74.047015)
811
(40.720513, -74.046644)
812
(40.721400, -74.045615)
813
(40.720955, -74.044720)
814
(40.722840, -74.047497)
815
(40.754005, -73.988745)
816
(40.754414, -73.988791)
817
(40.754227, -73.988613)
818
(40.752806, -73.

1283
(40.753651, -73.969484)
1284
(40.739110, -74.030474)
1285
(40.739955, -74.030216)
1286
(40.740080, -74.030934)
1287
(40.740107, -74.030884)
1288
(40.672970, -73.870974)
1289
(40.673022, -73.870991)
1290
(40.672931, -73.870968)
1291
(40.672954, -73.870894)
1292
(40.672953, -73.870853)
1293
(40.672955, -73.870842)
1294
(40.672956, -73.870842)
1295
(40.672956, -73.870836)
1296
(40.672982, -73.870842)
1297
(40.672988, -73.870842)
1298
(40.672975, -73.870779)
1299
(40.672938, -73.870750)
1300
(40.672948, -73.870744)
1301
(40.672983, -73.870710)
1302
(40.672895, -73.870767)
1303
(40.672896, -73.870721)
1304
(40.672876, -73.870698)
1305
(40.672856, -73.870698)
1306
(40.672852, -73.870681)
1307
(40.672840, -73.870727)
1308
(40.672843, -73.870664)
1309
(40.672882, -73.870767)
1310
(40.672874, -73.870819)
1311
(40.672889, -73.870888)
1312
(40.724241, -74.001313)
1313
(40.724285, -74.001964)
1314
(40.723927, -74.002271)
1315
(40.723185, -74.002547)
1316
(40.723308, -74.003407)
1317
(40.75854

1747
(40.748013, -73.987263)
1748
(40.748014, -73.987026)
1749
(40.747708, -73.986283)
1750
(40.746895, -73.985007)
1751
(40.746852, -73.984406)
1752
(40.761604, -73.986099)
1753
(40.761194, -73.987006)
1754
(40.762141, -73.985979)
1755
(40.762724, -73.984791)
1756
(40.762795, -73.984808)
1757
(40.764104, -73.986725)
1758
(40.764585, -73.987591)
1759
(40.764976, -73.987184)
1760
(40.764346, -73.988280)
1761
(40.764299, -73.988337)
1762
(40.757138, -73.968273)
1763
(40.757063, -73.967605)
1764
(40.757078, -73.967157)
1765
(40.756404, -73.967619)
1766
(40.757258, -73.968594)
1767
(40.920391, -74.000633)
1768
(40.931909, -74.012385)
1769
(40.694842, -73.994272)
1770
(40.693420, -73.994151)
1771
(40.694810, -73.994261)
1772
(40.697884, -73.992805)
1773
(40.698291, -73.996632)
1774
(40.733739, -74.002383)
1775
(40.733568, -74.001653)
1776
(40.734088, -74.002822)
1777
(40.734481, -74.002934)
1778
(40.746966, -73.981072)
1779
(40.745245, -73.982692)
1780
(40.765513, -73.983563)
1781
(40.76526

2226
(40.738263, -73.987838)
2227
(40.753014, -73.974351)
2228
(40.752896, -73.974144)
2229
(40.741819, -73.991402)
2230
(40.756787, -73.990145)
2231
(40.756731, -73.990007)
2232
(40.756862, -73.990007)
2233
(40.756988, -73.990099)
2234
(40.757244, -73.990060)
2235
(40.757480, -73.990335)
2236
(40.756928, -73.990759)
2237
(40.756903, -73.990920)
2238
(40.756774, -73.991811)
2239
(40.756636, -73.991675)
2240
(40.756531, -73.991591)
2241
(40.757464, -73.991809)
2242
(40.757839, -73.993095)
2243
(40.758253, -73.992986)
2244
(40.757377, -73.993353)
2245
(40.758253, -73.992481)
2246
(40.758161, -73.993898)
2247
(40.758196, -73.994162)
2248
(40.758295, -73.994369)
2249
(40.758283, -73.994616)
2250
(40.757634, -73.994983)
2251
(40.758095, -73.995677)
2252
(40.757623, -73.995477)
2253
(40.757707, -73.996423)
2254
(40.758064, -73.996423)
2255
(40.757998, -73.996596)
2256
(40.757998, -73.997020)
2257
(40.757998, -73.997365)
2258
(40.757414, -73.997020)
2259
(40.757228, -73.996974)
2260
(40.75675

2782
(40.769135, -73.955037)
2783
(40.767487, -73.953395)
2784
(40.773587, -73.959875)
2785
(40.767971, -73.981714)
2786
(40.768049, -73.981726)
2787
(40.768068, -73.981674)
2788
(40.768247, -73.981830)
2789
(40.768434, -73.981892)
2790
(40.752698, -73.990036)
2791
(40.751710, -73.990295)
2792
(40.752432, -73.990254)
2793
(40.753726, -73.988642)
2794
(40.741372, -73.989315)
2795
(40.742900, -73.992791)
2796
(40.725224, -74.007533)
2797
(40.725068, -74.007470)
2798
(40.724915, -74.007683)
2799
(40.725687, -74.005701)
2800
(40.733754, -74.006282)
2801
(40.732722, -74.004185)
2802
(40.733078, -74.003562)
2803
(40.732666, -74.003293)
2804
(40.733472, -74.003207)
2805
(40.733177, -74.005532)
2806
(40.731584, -74.005518)
2807
(40.730951, -74.004561)
2808
(40.730798, -74.004452)
2809
(40.732057, -74.007678)
2810
(40.728833, -74.005256)
2811
(40.727714, -74.007103)
2812
(40.736013, -74.001056)
2813
(40.722380, -74.003757)
2814
(40.722650, -74.004554)
2815
(40.722084, -73.996733)
2816
(40.72122

(40.779823, -73.953005)
3282
(40.784677, -73.955854)
3283
(40.780707, -73.958637)
3284
(40.717485, -73.962735)
3285
(40.717551, -73.962710)
3286
(40.719755, -73.957924)
3287
(40.721961, -73.958110)
3288
(40.679334, -73.968326)
3289
(40.678235, -73.968571)
3290
(40.679192, -73.971682)
3291
(40.677028, -73.972278)
3292
(40.680272, -73.974695)
3293
(40.679841, -73.974788)
3294
(40.680153, -73.974776)
3295
(40.676267, -73.971803)
3296
(40.674597, -73.981760)
3297
(40.676700, -73.980249)
3298
(40.760292, -73.987350)
3299
(40.760307, -73.987368)
3300
(40.760397, -73.987448)
3301
(40.761528, -73.985260)
3302
(40.718363, -73.990817)
3303
(40.718690, -73.989313)
3304
(40.719370, -73.989577)
3305
(40.720737, -73.988760)
3306
(40.741933, -73.986976)
3307
(40.732254, -74.002001)
3308
(40.732626, -74.001973)
3309
(40.733102, -74.003006)
3310
(40.706858, -74.006087)
3311
(40.707396, -74.006265)
3312
(40.706060, -74.006426)
3313
(40.705892, -74.005703)
3314
(40.712671, -74.011931)
3315
(40.712046, -7

3781
(40.746965, -74.000897)
3782
(40.747896, -74.000372)
3783
(40.746563, -74.001617)
3784
(40.769324, -73.982824)
3785
(40.841184, -73.939854)
3786
(40.841268, -73.940262)
3787
(40.841292, -73.940463)
3788
(40.840535, -73.941207)
3789
(40.718827, -73.988210)
3790
(40.719879, -73.987716)
3791
(40.718700, -73.987700)
3792
(40.620475, -73.964393)
3793
(40.620412, -73.964479)
3794
(40.620146, -73.965064)
3795
(40.747333, -73.975017)
3796
(40.747556, -73.974959)
3797
(40.747817, -73.974862)
3798
(40.764070, -73.954262)
3799
(40.764665, -73.954811)
3800
(40.763988, -73.953476)
3801
(40.764557, -73.953487)
3802
(40.763174, -73.954164)
3803
(40.714826, -74.005692)
3804
(40.714812, -74.005932)
3805
(40.712035, -74.007817)
3806
(40.724238, -73.997657)
3807
(40.723933, -73.998042)
3808
(40.723868, -73.998208)
3809
(40.723400, -73.999402)
3810
(40.727101, -74.000240)
3811
(40.728528, -74.007482)
3812
(40.751189, -73.990639)
3813
(40.745878, -73.988142)
3814
(40.743939, -73.983999)
3815
(40.74260

4127
(40.758977, -73.985118)
4128
(40.761653, -73.984462)
4129
(40.760937, -73.987507)
4130
(40.743442, -73.921346)
4131
(40.743578, -73.923339)
4132
(40.744144, -73.923386)
4133
(40.743926, -73.923773)
4134
(40.741014, -73.922496)
4135
(40.657535, -73.735911)
4136
(40.657535, -73.736359)
4137
(40.657603, -73.735957)
4138
(40.657568, -73.736250)
4139
(40.770792, -73.985925)
4140
(40.770853, -73.984865)
4141
(40.770402, -73.984637)
4142
(40.848254, -74.118123)
4143
(40.860722, -74.108709)
4144
(40.867988, -74.118123)
4145
(40.859775, -74.129868)
4146
(40.780627, -73.802340)
4147
(40.781166, -73.802627)
4148
(40.782323, -73.803603)
4149
(40.755957, -73.755735)
4150
(40.755677, -73.755641)
4151
(40.755621, -73.755497)
4152
(40.755406, -73.755333)
4153
(40.755956, -73.756097)
4154
(40.755789, -73.756247)
4155
(40.756197, -73.756199)
4156
(40.756415, -73.755936)
4157
(40.756601, -73.756764)
4158
(40.716189, -73.998158)
4159
(40.716070, -73.997380)
4160
(40.715550, -73.998456)
4161
(40.71493

In [20]:
########################################dsr plot section############################################
import numpy as np
import matplotlib.pyplot as plt
x = []
y = []
colors = []
for venue in venues:
    if dsr_dict[venues.index(venue)] < 2.1:
        x.append(venue.longitude)
        y.append(venue.latitude)
        colors.append(dsr_dict[venues.index(venue)])
        
plt.figure(figsize=(20, 20), dpi=80)
plt.scatter(x, y, c=colors, alpha=0.5, vmin=0, vmax=2, cmap='viridis')
plt.colorbar()
plt.savefig("dsr_value_new.png")

In [21]:
#########################################csv file##########################################################
import csv
my_dict = dsr_dict
with open('dsr_value.csv', 'w') as f:
    for key in my_dict.keys():
        f.write("%s,%s\n"%(key,my_dict[key]))